In [1]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import pandas as pd
import json

# ----------- CONFIGURE THESE PATHS -----------
BASE_DIR = Path("/Users/singh/Documents/Kognition/Matlab/action-analysis/app_processing/WinGaze")
DATA_DIR = BASE_DIR / "data"

# Input files
JSON_PATH = DATA_DIR / "vp34_info.player.json"
FIXATION_FILES = {
    "robot": DATA_DIR / "fixations_on_surface_robot.csv",
    "task": DATA_DIR / "fixations_on_surface_task.csv",
}

# Output root (can be same as input)
FIX_ROOT_OUT = DATA_DIR

In [2]:
def load_json_meta(json_path: Path):
    with open(json_path, "r") as f:
        meta = json.load(f)
    # Pupil definitions:
    # - start_time_system_s: UNIX time (seconds) at recording start
    # - start_time_synced_s: Pupil's synced time (seconds) at recording start
    start_time_system_s = float(meta["start_time_system_s"])
    start_time_synced_s = float(meta["start_time_synced_s"])
    # Offset to convert Pupil's world_timestamp -> UNIX seconds
    t_offset = start_time_system_s - start_time_synced_s
    return start_time_system_s, start_time_synced_s, t_offset

In [3]:
def process_surface_csv(csv_in: Path, start_time_system_s: float, t_offset: float) -> pd.DataFrame:
    df = pd.read_csv(csv_in)

    if "world_timestamp" not in df.columns:
        raise ValueError(f"Missing 'world_timestamp' in {csv_in}")

    # (a) Sample timestamps (per row)
    df["start_timestamp_unix"] = df["world_timestamp"] + t_offset

    # Human-readable (optional)
    df["start_timestamp_datetime_utc"] = pd.to_datetime(df["start_timestamp_unix"], unit="s", utc=True)

    # Elapsed since experiment start
    df["Total_time_sec"] = df["start_timestamp_unix"] - start_time_system_s
    df["Total_time_ms"]  = df["Total_time_sec"] * 1000.0

    # Clean tiny negatives due to float imprecision
    df.loc[df["Total_time_ms"].between(-1e-3, 0), ["Total_time_ms", "Total_time_sec"]] = [0.0, 0.0]

    # (b) Fixation start/end from start_timestamp + duration (if present)
    if "start_timestamp" in df.columns:
        df["fixation_start_unix"] = df["start_timestamp"] + t_offset
        df["fixation_start_ms"] = (df["fixation_start_unix"] - start_time_system_s) * 1000.0
        df.loc[df["fixation_start_ms"].between(-1e-3, 0), "fixation_start_ms"] = 0.0
        if "duration" in df.columns:
            df["fixation_duration_ms"] = df["duration"]
            df["fixation_end_ms"] = df["fixation_start_ms"] + df["fixation_duration_ms"]

    # (c) Duration per fixation_id using max - min on the experiment clock (ms)
    if "fixation_id" not in df.columns:
        # If not present, we cannot compute per-fixation durations
        df["duration_ms"] = pd.NA
    else:
        df["duration_ms"] = (
            df.groupby("fixation_id")["Total_time_ms"]
              .transform(lambda s: float(s.max() - s.min()))
        )

    return df

In [4]:
def main():
    if not JSON_PATH.exists():
        print(f"[WARN] Missing JSON: {JSON_PATH}")
        return

    try:
        start_time_system_s, start_time_synced_s, t_offset = load_json_meta(JSON_PATH)
    except Exception as e:
        print(f"[WARN] Failed to read JSON: {e}")
        return

    for label, csv_in in FIXATION_FILES.items():
        if not csv_in.exists():
            print(f"[WARN] Missing CSV for {label}: {csv_in}")
            continue

        try:
            df = process_surface_csv(csv_in, start_time_system_s, t_offset)
        except Exception as e:
            print(f"[WARN] Failed to process {csv_in}: {e}")
            continue

        csv_out = FIX_ROOT_OUT / f"{csv_in.stem}_with_time.csv"
        csv_out.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(csv_out, index=False)

        # Quick feedback
        nfix = df["fixation_id"].nunique() if "fixation_id" in df.columns else 0
        trial_len_sec = float(df["Total_time_sec"].max()) if len(df) else 0.0
        print(f"Saved: {csv_out} | Fixations: {nfix} | Trial length: {trial_len_sec:.3f}s")

In [5]:
if __name__ == "__main__":
    main()

Saved: /Users/singh/Documents/Kognition/Matlab/action-analysis/app_processing/WinGaze/data/fixations_on_surface_robot_with_time.csv | Fixations: 5819 | Trial length: 2786.173s
Saved: /Users/singh/Documents/Kognition/Matlab/action-analysis/app_processing/WinGaze/data/fixations_on_surface_task_with_time.csv | Fixations: 5616 | Trial length: 2786.173s


In [5]:
def process_pupil_positions_csv(csv_in: Path,
                                start_time_system_s: float,
                                t_offset: float) -> pd.DataFrame:
    """
    Align pupil_positions.csv timestamps using the JSON meta info.

    Parameters
    ----------
    csv_in : Path
        Path to pupil_positions.csv for one participant.
    start_time_system_s : float
        UNIX time (seconds) at experiment start (from JSON).
    t_offset : float
        Offset to convert world_timestamp -> UNIX seconds
        (start_time_system_s - start_time_synced_s).

    Returns
    -------
    pd.DataFrame
        Original data with added columns:
        - start_timestamp_unix
        - start_timestamp_datetime_utc
        - Total_time_sec
        - Total_time_ms
    """
    df = pd.read_csv(csv_in)

    if "pupil_timestamp" not in df.columns:
        raise ValueError(f"Missing 'pupil_timestamp' in {csv_in}")

    # (a) Experiment-clock timestamps
    # Absolute UNIX time for each row
    df["start_timestamp_unix"] = df["pupil_timestamp"] + t_offset

    # Human-readable (optional)
    df["start_timestamp_datetime_utc"] = pd.to_datetime(
        df["start_timestamp_unix"],
        unit="s",
        utc=True
    )

    # Elapsed since experiment start
    df["Total_time_sec"] = df["start_timestamp_unix"] - start_time_system_s
    df["Total_time_ms"]  = df["Total_time_sec"] * 1000.0

    # Clean tiny negatives due to float imprecision
    df.loc[df["Total_time_ms"].between(-1e-3, 0), ["Total_time_ms", "Total_time_sec"]] = [0.0, 0.0]

    return df


In [10]:
# -------- CONFIGURE THESE PATHS --------
BASE_DIR = Path("/Users/singh/Documents/Kognition/Matlab")

PUPIL_ROOT_IN  = BASE_DIR / "negation_data_analysis/HRI-analysis/fixation_positions"
JSON_ROOT      = BASE_DIR / "data_pupil_pro/data_processed/jsonfiles"
PUPIL_ROOT_OUT = BASE_DIR / "negation_data_analysis/HRI-analysis/fixation_positions/"  # or choose another output dir

# Subjects to process (same as in your notebook or adjust)
vp_valid = (13, 14, 15, 16, 17, 19, 20, 21, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34)


def main_pupil_positions():
    for subj in vp_valid:
        json_path = JSON_ROOT / f"vp{subj}_info.player.json"
        if not json_path.exists():
            print(f"[WARN] Missing JSON for vp{subj}: {json_path}")
            continue

        try:
            start_time_system_s, start_time_synced_s, t_offset = load_json_meta(json_path)
        except Exception as e:
            print(f"[WARN] Failed to read JSON for vp{subj}: {e}")
            continue

        # Adjust this path pattern to your actual folder structure
        csv_in  = PUPIL_ROOT_IN  / f"vp{subj}/surfaces/pupil/pupil_positions.csv"
        csv_out = PUPIL_ROOT_OUT / f"vp{subj}/surfaces/pupil/pupil_positions_with_time.csv"

        if not csv_in.exists():
            print(f"[WARN] Missing pupil_positions for vp{subj}: {csv_in}")
            continue

        try:
            df = process_pupil_positions_csv(csv_in, start_time_system_s, t_offset)
        except Exception as e:
            print(f"[WARN] Failed to process {csv_in}: {e}")
            continue

        csv_out.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(csv_out, index=False)

        # Quick feedback
        n_rows = len(df)
        trial_len_sec = float(df["Total_time_sec"].max()) if n_rows else 0.0
        print(f"Saved: {csv_out} | Rows: {n_rows} | Trial length: {trial_len_sec:.3f}s")


if __name__ == "__main__":
    main_pupil_positions()


Saved: /Users/singh/Documents/Kognition/Matlab/negation_data_analysis/HRI-analysis/fixation_positions/vp13/surfaces/pupil/pupil_positions_with_time.csv | Rows: 1683042 | Trial length: 3395.289s
Saved: /Users/singh/Documents/Kognition/Matlab/negation_data_analysis/HRI-analysis/fixation_positions/vp14/surfaces/pupil/pupil_positions_with_time.csv | Rows: 1579604 | Trial length: 3186.759s
Saved: /Users/singh/Documents/Kognition/Matlab/negation_data_analysis/HRI-analysis/fixation_positions/vp15/surfaces/pupil/pupil_positions_with_time.csv | Rows: 1454670 | Trial length: 2935.263s
Saved: /Users/singh/Documents/Kognition/Matlab/negation_data_analysis/HRI-analysis/fixation_positions/vp16/surfaces/pupil/pupil_positions_with_time.csv | Rows: 1401286 | Trial length: 2826.924s
Saved: /Users/singh/Documents/Kognition/Matlab/negation_data_analysis/HRI-analysis/fixation_positions/vp17/surfaces/pupil/pupil_positions_with_time.csv | Rows: 1794092 | Trial length: 3619.513s
Saved: /Users/singh/Documents/